In [160]:
from Bio.PDB import Select, PDBIO
from Bio.PDB.PDBParser import PDBParser
class ChainSelect(Select):
    def __init__(self, chain):
        self.chain = chain
    def accept_chain(self, chain):
        if chain.get_id() == self.chain:
            return 1
        else:          
            return 0
chains = ['A','B','C','D','E','S']
p = PDBParser(PERMISSIVE=1)       
structure = p.get_structure("ccr", "ccr_b7.pdb")
for chain in chains:
    pdb_chain_file = 'pdb_file_chain_{}.pdb'.format(chain)                                 
    io_w_no_h = PDBIO()               
    io_w_no_h.set_structure(structure)
    io_w_no_h.save('{}'.format(pdb_chain_file), ChainSelect(chain))

In [161]:
class HisRenamer:
    def __init__(self, path=None):
        """Creates an object based on a pdb file located in path. This object can modify HIS names according 
        to protonation and write a new corrected file."""
        self.path = path
        
    def HIS_detector(self):
        HDList=[]
        HEList=[]
        temp1=PDBParser(PERMISSIVE=1)
        temp2 = temp1.get_structure("temp3",self.path)
        for model in temp2:
            for chain in model:
                for residue in chain.get_residues():
                    if residue.get_resname()=="HIS":
                        if residue.has_id("HD1"):
                            HDList.append(str(residue.get_id()[1]))
                        else:
                            HEList.append(str(residue.get_id()[1]))
                    pass
        return (HDList,HEList)
                    
    def LineContainer(self):
        with open(self.path,'r') as temp1:
            temp2=temp1.readlines()
        return temp2
    
    def HisChanger(self):
        temp1=self.LineContainer()
        temp2=self.HIS_detector()
        temp3=[]
        for line in temp1:
            if line.startswith('ATOM'):
                if line.split()[5] in temp2[0]:
                    newline=line.replace('HIS','HSD')
                    temp3.append(newline)
                elif line.split()[5] in temp2[1]:
                    newline=line.replace('HIS','HSE')
                    temp3.append(newline)
                else:
                    temp3.append(line)
            else: 
                pass
        temp3.append("END\n")
        return temp3
    
    def HisWriter(self, out_path):
        temp1=self.HisChanger()
        with open(out_path,'w') as temp2:
            for line in temp1:
                temp2.write(line)
        
    
        
    

In [162]:
for letra in chains[0:4]:
    a=HisRenamer("pdb_file_chain_"+letra+".pdb")
    a.HisWriter("correctedChain"+letra+".pdb")